
# **Gestión de datos y datos digitales**

## **Actividad individual 3: "Películas recomendadas en Amazon"**


### Autor: Fabio Santamaría Iglesias



15/03/2021

## **Introducción:**

Preparamos el entorno instalando e importando los paquetes adecuados:
    
    

In [1]:
from IPython import get_ipython
from IPython import display
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  !apt-get update
  !apt install chromium-chromedriver
  !cp /usr/lib/chromium-browser/chromedriver /usr/bin
  !pip install selenium
  !pip install numpy
  !pip install pandas
  !pip install cssselect

Abrimos el navegador desde Selenium:

In [2]:
import numpy as np
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
if 'google.colab' in str(get_ipython()):
  # instalar drivers
  options = webdriver.ChromeOptions()
  options.add_argument('-headless')
  options.add_argument('-no-sandbox')
  options.add_argument('-disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',options=options)
else:
  import os 
  os.environ["PATH"] = '$PATH:.'
  options = webdriver.ChromeOptions()
  driver = webdriver.Chrome()

Accedemos a la página de Amazon y aceptamos las cookies si las hubiera.

In [3]:
driver.get("https://www.amazon.es/")

driver.implicitly_wait(10)

try:
    driver.find_element(By.ID,'sp-cc-accept').click()
except:
    print('No hay cookies que aceptar')

Buscamos la caja de búsqueda por ID, borramos el contenido, introducimos la nuestra película favorita y simulamos enter para lanzar la petición.

In [4]:
driver.find_element(By.ID, "twotabsearchtextbox").clear()

#INCLUIR SIEMPRE LA PALABRA "DVD"
driver.find_element(By.ID, "twotabsearchtextbox").send_keys("El señor de los anillos dvd")

driver.find_element(By.ID, "twotabsearchtextbox").send_keys(Keys.ENTER)

driver.implicitly_wait(10)

Buscamos el primer resultado por XPATH. Estamos buscando la primera caja de texto del resultado de la búsqueda que además contenga la palabra "DVD". Hacemos click sobre el elemento, que nos conduce a la página de nuestra película favorita.

In [35]:
driver.find_element(By.XPATH,'(//span[@class="a-size-base-plus a-color-base a-text-normal" and contains(text(), \"DVD\")])').click()

Capturamos el código html con la librería lxml. Podríamos cerrar la página ahora y continuar con el código capturado. No obstante, dejamos esto para el final para poder ver que la información obtenida es la correcta.

In [36]:
import lxml

from lxml import html
from lxml import cssselect

In [37]:
html_text = driver.page_source
tree = html.document_fromstring(html_text)

Con document_fromstring convertimos el texto html a un árbol de nodos. De aquí podemos sacar la información. Por ejemplo, el título del producto seleccionado, en este caso, nuestra película favorita:

In [38]:
prod = tree.xpath('//*[@id="productTitle"]')[0]
print(prod.text_content().strip())

Trilogía El Señor De Los Anillos Cinematográfica [DVD]


También podemos seleccionar los productos comprados juntos habitualmente con cssselect.

In [39]:
recomended_products_name_list = tree.cssselect('#sims-fbt-form > div.sims-fbt-rows > fieldset > ul > li:nth-child(n) > span > span > div > label > span > div > a')
prod_list = []
for name in recomended_products_name_list:
      prod_list.append(name.text_content())
      print(name.text_content() + "\n")

Trilogia El Hobbit Extendida [DVD]

Pack Harry Potter - Colección Completa Golden Steelbook 2019 [DVD]



In [40]:
recomended_products_price_list = tree.cssselect('#sims-fbt-form > div.sims-fbt-rows > fieldset > ul > li:nth-child(n+2) > span > span > div > label > span > div > span.a-color-price > span')
price_list = []
for price in recomended_products_price_list:
      price_list.append(price.text_content())
      print(price.text_content() + "\n")

50,58 €

39,99 €



Por último, podemos convertir la información capturada en un dataframe.

In [41]:
if prod_list:
    display.display(pd.DataFrame({'products' : prod_list, 'price': price_list}))
else:
    print("Este producto no tiene productos comprados juntos habitualmente")

,products,price
0,Trilogia El Hobbit Extendida [DVD],"50,58 €"
1,Pack Harry Potter - Colección Completa Golden ...,"39,99 €"


Una vez que hemos comprobado que la informarción capturada es correcta, cerramos la página:

In [42]:
#driver.quit()

Es posible repetir el proceso para otras películas. Algunas con las que se ha probado son las siguientes:

*  El Rey León DVD
*  Interstellar DVD
*  Odisea 2001 DVD
*  El viaje de Chihiro DVD
*  El señor de los anillos DVD
*  Star Wars I DVD

En todas ellas se ha verificado que los resultados son correctos. 